In [2]:
from idxutils import gen_in_network_links, JSONOpen
from tqdm import tqdm
import pandas as pd
import requests
import sys
from _utils import istarmap
import aiohttp
import asyncio
from idxutils import gen_in_network_links
import traceback as tb

'''
Get size of each file in MRF (step 2)
'''

'\nGet size of each file in MRF (step 2)\n'

In [10]:
async def get_network_files(session, url):
    try:
        async with session.head(url) as r:
            print(r.status)
            size = r.headers.get("Content-Length")
            return url, size

                    
    except aiohttp.ClientResponseError as e:
        print(e)
        return url, None


    except asyncio.TimeoutError:
        return url, None

    except aiohttp.ClientConnectionError:
        return url, None

    except KeyboardInterrupt:
        import os; os.exit()

    # except UnicodeDecodeError:
    #     tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return url, None

In [14]:
from tqdm.asyncio import tqdm

# Not entirely sure why I wrote two methods

# Pandas
async def main(file):
    df = pd.read_csv(file)
    df1 = pd.DataFrame(columns=["url", "size"])
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in df["url"]), leave=True, position=0)
        return df1

# File
async def main_file():
    df1 = pd.DataFrame(columns=["url", "size"])
    with open(file, "r") as f:
        total = len(f.readlines())
        f.seek(0)
        async with aiohttp.ClientSession(raise_for_status = True) as session:
            df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in f), leave=True, position=0)
            return df1

In [15]:
t = await main("fidelis_test.csv")

100%|██████████| 2/2 [00:00<00:00,  5.92it/s]

301
301


In [6]:
t

,url,size
0,http://centene.com/content/dam/centene/Centene...,None
1,http://centene.com/content/dam/centene/Centene...,None
2,http://centene.com/content/dam/centene/Centene...,None
3,http://centene.com/content/dam/centene/Centene...,None
4,http://centene.com/content/dam/centene/Centene...,None
5,http://centene.com/content/dam/centene/Centene...,None
6,http://centene.com/content/dam/centene/Centene...,None
7,http://centene.com/content/dam/centene/Centene...,None
8,http://centene.com/content/dam/centene/Centene...,None
9,http://centene.com/content/dam/centene/Centene...,None


In [24]:
t.to_csv("UHC_payers.csv", index=False)